2 Analyzing GMO

In [74]:
import numpy as np
import pandas as pd
import portfolio_management_helper as pmh

rf_rts = pd.read_excel("../data/gmo_data.xlsx",sheet_name="risk-free rate").set_index("date")
gmo_rts = pd.read_excel("../data/gmo_data.xlsx",sheet_name="total returns").set_index("date")
excess_rts = gmo_rts.subtract(rf_rts["TBill 3M"],axis=0)

excess_rts.tail()

,SPY,GMWAX,GMGEX
date,,,
2024-06-28,-0.0183,-0.0607,-0.0666
2024-07-31,-0.0407,-0.0224,-0.0182
2024-08-30,-0.0278,-0.0364,-0.0316
2024-09-30,-0.0252,-0.0337,-0.0308
2024-10-31,-0.0543,-0.0823,-0.0849


In [75]:
pmh.calc_summary_statistics(
    returns=gmo_rts[["GMWAX"]],
    annual_factor=12,
    provided_excess_returns=True,
    timeframes={
        "1927-2011": ["1927", "2011"],
        "2012-2024": ["2012", "2024"],
        "1927-2024": ["1927", "2024"],
    },
    keep_columns=["Annualized Mean", "Annualized Vol", "Annualized Sharpe"],
)

,Annualized Mean,Annualized Vol,Annualized Sharpe
GMWAX 1927-2011,0.0748,0.1103,0.6775
GMWAX 2012-2024,0.0570,0.0951,0.5997
GMWAX 1927-2024,0.0666,0.1035,0.6436


The mean and the Sharpe both dipped from 2012 - 2024, but the volatiity improved somewhat during that time period, too. Overall, these factors are somewhat consistent through the different time frames.

In [76]:
pmh.calc_summary_statistics(
    returns=gmo_rts[["GMWAX"]],
    annual_factor=12,
    provided_excess_returns=True,
    var_quantile = .05,
    timeframes={
        "1927-2011": ["1927", "2011"],
        "2012-2024": ["2012", "2024"],
        "1927-2024": ["1927", "2024"],
    },
    keep_columns=["Min", "Annualized Historical VaR", "Max Drawdown"],
)


,Min,Annualized Historical VaR (5.00%),Max Drawdown
GMWAX 1927-2011,-0.1451,-0.1524,-0.2936
GMWAX 2012-2024,-0.1150,-0.1322,-0.2168
GMWAX 1927-2024,-0.1451,-0.1405,-0.2936


The GMWAX does seem to have fairly high tail risk, considering that the max drawdown is nearly 30%, and the 5th percentile VaR is 14%. This does improv a bit in 2012-2024 versus 1927-2011.

In [77]:
pmh.calc_regression(
    y = gmo_rts[["GMWAX"]],
    X = gmo_rts[["SPY"]],
    annual_factor=12,
    timeframes={
        "1927-2011": ["1927", "2011"],
        "2012-2024": ["2012", "2024"],
        "1927-2024": ["1927", "2024"],
    },
    keep_columns=["Annualized Alpha", "Beta", "R-Squared"],
)

"calc_regression" assumes excess returns to calculate Information and Treynor Ratios


,Annualized Alpha,R-Squared,SPY Beta
GMWAX 1927-2011,0.0400,0.6501,0.5414
GMWAX 2012-2024,-0.0284,0.7485,0.5824
GMWAX 1927-2024,0.0102,0.6796,0.5527


The GMWAX is a low beta strategy, as its beta is under 1 in all the time series. However, the beta does grow a bit higher from 2012 onwards. 

The GMWAX does provide alpha on average, although since 2012 that was deteorated significantly, where it has since actually since provided a negative alpha.

In [78]:
pmh.calc_summary_statistics(
    returns=gmo_rts[["GMGEX"]],
    annual_factor=12,
    provided_excess_returns=True,
    timeframes={
        "1927-2011": ["1927", "2011"],
        "2012-2024": ["2012", "2024"],
        "1927-2024": ["1927", "2024"],
    },
    keep_columns=["Annualized Mean", "Annualized Vol", "Annualized Sharpe"],
)

,Annualized Mean,Annualized Vol,Annualized Sharpe
GMGEX 1927-2011,0.0245,0.1471,0.1667
GMGEX 2012-2024,0.0149,0.2359,0.0632
GMGEX 1927-2024,0.0201,0.1928,0.1043


In [79]:
pmh.calc_summary_statistics(
    returns=gmo_rts[["GMGEX"]],
    annual_factor=12,
    provided_excess_returns=True,
    var_quantile = .05,
    timeframes={
        "1927-2011": ["1927", "2011"],
        "2012-2024": ["2012", "2024"],
        "1927-2024": ["1927", "2024"],
    },
    keep_columns=["Min", "Annualized Historical VaR", "Max Drawdown"],
)

,Min,Annualized Historical VaR (5.00%),Max Drawdown
GMGEX 1927-2011,-0.1512,-0.2759,-0.5556
GMGEX 2012-2024,-0.6587,-0.2356,-0.7374
GMGEX 1927-2024,-0.6587,-0.2622,-0.7618


In [80]:
pmh.calc_regression(
    y = gmo_rts[["GMGEX"]],
    X = gmo_rts[["SPY"]],
    annual_factor=12,
    timeframes={
        "1927-2011": ["1927", "2011"],
        "2012-2024": ["2012", "2024"],
        "1927-2024": ["1927", "2024"],
    },
    keep_columns=["Annualized Alpha", "Beta", "R-Squared"],
)

"calc_regression" assumes excess returns to calculate Information and Treynor Ratios


,Annualized Alpha,R-Squared,SPY Beta
GMGEX 1927-2011,-0.0244,0.7263,0.7628
GMGEX 2012-2024,-0.1086,0.2543,0.8423
GMGEX 1927-2024,-0.0603,0.3983,0.7880


The GMGEX performs worse across the board: it has worse Sharpe, worse drawdowns, and offers less alpha/higher correlation with SPY.

3 Forecast Regressions

In [81]:
signals = pd.read_excel("../data/gmo_data.xlsx",sheet_name="signals").set_index("date")
signals.tail()

,SPX DVD YLD,SPX P/E,TNote 10YR
date,,,
2024-06-28,1.3271,25.5386,4.3961
2024-07-31,1.3205,25.7965,4.0296
2024-08-30,1.2990,25.5680,3.9034
2024-09-30,1.2789,26.1183,3.7809
2024-10-31,1.3046,25.8491,4.2844


In [82]:
pmh.calc_regression(
    y = gmo_rts[["SPY"]],
    X = signals[["SPX DVD YLD"]].shift(-1),
    annual_factor=12,
    keep_columns=["Annualized Alpha", "Beta", "R-Squared"],
)

"calc_regression" assumes excess returns to calculate Information and Treynor Ratios


,Annualized Alpha,R-Squared,SPX DVD YLD Beta
SPY,0.4271,0.0180,-0.0149


In [83]:
pmh.calc_regression(
    y = gmo_rts[["SPY"]],
    X = signals[["SPX P/E"]].shift(-1),
    annual_factor=12,
    keep_columns=["Annualized Alpha", "Beta", "R-Squared"],
)

"calc_regression" assumes excess returns to calculate Information and Treynor Ratios


,Annualized Alpha,R-Squared,SPX P/E Beta
SPY,-0.2480,0.0205,0.0015


In [84]:
pmh.calc_regression(
    y = gmo_rts[["SPY"]],
    X = signals[["TNote 10YR"]].shift(-1),
    annual_factor=12,
    keep_columns=["Annualized Alpha", "Beta", "R-Squared"],
)

"calc_regression" assumes excess returns to calculate Information and Treynor Ratios


,Annualized Alpha,R-Squared,TNote 10YR Beta
SPY,0.1107,0.0000,-0.0002


In [85]:
pmh.calc_regression(
    y = gmo_rts[["SPY"]],
    X = signals[["SPX DVD YLD","SPX P/E","TNote 10YR"]].shift(-1),
    annual_factor=12,
    keep_columns=["Annualized Alpha", "Beta", "R-Squared"],
)

"calc_regression" assumes excess returns to calculate Information and Treynor Ratios


,Annualized Alpha,R-Squared,SPX DVD YLD Beta,SPX P/E Beta,TNote 10YR Beta
SPY,0.2477,0.0266,-0.0116,0.0009,-0.0023
